In [1]:
%matplotlib inline

import pandas as pd
import watexr_forecast as wx
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter('ignore')

# WATExR: Seasonal forecast workflow

## 1. Period of interest

Specify the year and season of interest in the cell below.

In [2]:
# Year of interest
year = 2001

# Season of interest. Choose from:
# ['winter', 'spring', 'early_summer', 'late_summer']
season = 'early_summer'

# Path to historic (EWEMBI) dataset for 1981 - 2010
ewembi_path = r'../../Data/Meteorological/01_ewembi_obs/ewembi_obs_1981-2010.dat'

# CSV to create summarising EWEMBI quantiles (optional - see Section 3)
quant_path = r'../../Data/Meteorological/01_ewembi_obs/ewembi_obs_quantiles_1981-2010.csv'

## 2. Get seasonal forecast 

Download and bias-correct (without cross-validation) the 15-member seasonal forecast for the specificed year and season.

**Note:** Takes several minutes to complete.

**Debugging:** If the call below fails, trying opening a terminal and running:

    cd projects/WATExR/Norway_Morsa/MetData_Processing/notebooks
    
then

    Rscript --vanilla get_seasonal_forecast.R 2001 5,6,7
    
This will give more informative errors.

In [3]:
%%time

# Download and bias correct S4 data
res = wx.get_seasonal_forecast(year, season)

CalledProcessError: Command '['Rscript', '--vanilla', 'get_seasonal_forecast.R', '2001', '5,6,7']' returned non-zero exit status 1.

## 3. (Re-)Calculate historic EWEMBI quantiles (optional)

**Note:** This cell only needs to be re-run if the EWEMBI dataset is changed or updated.

In [ ]:
# Summarise EWEMBI data
wx.calculate_ewembi_quantiles(ewembi_path, 
                              quant_path, 
                              quants=[0.05, 0.33, 0.67, 0.95],
                              names=['date', 'time', 'uas', 'vas', 'ps', 'tas', 'pr', 'hurs', 'petH'],
                             )

## 4. Aggregate seasonal forecast data

In [ ]:
# Average seasonal data
s4_df = wx.aggregate_seasonal_forecast(season, 
                                       par_list=['tas', 'pr', 'wind'],
                                       names=['date', 'time', 'uas', 'vas', 'ps', 'tas', 'pr', 'hurs', 'petH'],
                                      )
s4_df

## 5. Compare S4 forecast to historic EWEMBI quantiles

Classifies the S4 forecast for this season as "Extreme low", "Below normal", " Near normal", "Above normal" or "Extreme high", based on historic EWEMBI quantiles.

In [ ]:
# Classify forecast
res_dict = wx.compare_s4_to_ewembi(quant_path, 
                                   season, 
                                   s4_df, 
                                   par_list=['tas', 'pr', 'wind'],
                                   normal_quants=[0.33, 0.67], 
                                   extreme_quants=[0.05, 0.95],
                                  )

res_dict

## 6. Make forecast PNG

**Note:** The forecast reliability score for "wind" is currently based on the ROCSS score for `uas` **only**. Need to fix this.

In [ ]:
# Read table of historic performance
xl_path = r'../../Data/Meteorological/04_tercile_plots/seasonal_forecast_performance_1981-2010.xlsx'
perf_df = pd.read_excel(xl_path)

# Make summary image
forecast_png = r'forecast_output/climate_forecast_summary.png'
wx.make_climate_forecast_png(season, res_dict, perf_df, forecast_png)

## 7. Make forecast PDF

In [ ]:
# Export PDF
wx.make_forecast_pdf(year, season)